DATA EXPLORATION USING CLASSICMODEL DATABASE

In [1]:
import mysql.connector as cn
import pandas as pd

In [2]:
mydb = cn.connect(
    host = 'localhost',
    user = 'root',
    passwd = input('Input Password MySQL : '),
    database = input('Choose the database : ')
)

In [3]:
mycursor = mydb.cursor()

def query (x) :
    mycursor.execute (x)
    result = mycursor.fetchall()
    df = pd.DataFrame(result, columns = mycursor.column_names)
    return df

In [4]:
query(
    '''
    show tables    
    '''
)

,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


In [18]:
#to find out customers distribution 
query(
    '''
    SELECT city, count(customername) AS qty_cust
    FROM customers 
    GROUP BY city 
    ORDER BY qty_cust desc;
    '''
)

,city,qty_cust
0,NYC,5
1,Madrid,5
2,Singapore,3
3,Auckland,3
4,Brickhaven,3
...,...,...
90,Mannheim,1
91,Saint Petersburg,1
92,Herzlia,1
93,Sevilla,1


In [6]:
# to find out if there is someone who followed up by director --> no one

query(
    '''
    SELECT * 
    FROM customers 
    WHERE salesRepEmployeeNumber = "1002";
    '''
)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit


In [7]:
# try self join

query(
    '''
    SELECT e.employeenumber, e.firstname, m.firstname AS manager
    FROM employees e
    JOIN employees m
	ON e.reportsTo = m.employeenumber;
    '''
)

,employeenumber,firstname,manager
0,1056,Mary,Diane
1,1076,Jeff,Diane
2,1088,William,Mary
3,1102,Gerard,Mary
4,1143,Anthony,Mary
5,1165,Leslie,Anthony
6,1166,Leslie,Anthony
7,1188,Julie,Anthony
8,1216,Steve,Anthony
9,1286,Foon Yue,Anthony


In [8]:
# coba left join

query(
    '''
    SELECT e.employeenumber, e.firstname, m.firstname as manager
    FROM employees e
    LEFT JOIN employees m
	ON e.reportsTo = m.employeenumber;
    '''
)

,employeenumber,firstname,manager
0,1002,Diane,None
1,1056,Mary,Diane
2,1076,Jeff,Diane
3,1088,William,Mary
4,1102,Gerard,Mary
5,1143,Anthony,Mary
6,1165,Leslie,Anthony
7,1166,Leslie,Anthony
8,1188,Julie,Anthony
9,1216,Steve,Anthony


In [9]:
# u/ tahu banyaknya penjualan di setiap cabang

query(
    '''
    SELECT 
	o.city,
    pl.productline, 
    count(distinct od.productCode) as "Banyaknya Produk"
    FROM productlines pl
    LEFT JOIN products p ON pl.productline = p.productline
    LEFT JOIN orderdetails od ON p.productCode = od.productCode
    LEFT JOIN orders ord ON od.orderNumber = ord.orderNumber
    JOIN customers c ON ord.customerNumber = c.customerNumber
    JOIN employees e ON c.salesrepemployeeNumber = e.employeeNumber
    JOIN offices o ON e.officecode = o.officecode
    GROUP BY o.officecode, pl.productline;
    '''
)

,city,productline,Banyaknya Produk
0,San Francisco,Classic Cars,37
1,San Francisco,Motorcycles,13
2,San Francisco,Planes,12
3,San Francisco,Ships,9
4,San Francisco,Trains,3
5,San Francisco,Trucks and Buses,11
6,San Francisco,Vintage Cars,24
7,Boston,Classic Cars,36
8,Boston,Motorcycles,13
9,Boston,Planes,10


In [10]:
query(
    '''
    SELECT *
    FROM orders o
    JOIN customers c
	ON o.customernumber = c.customernumber
    JOIN orderdetails od
	ON o.ordernumber = o.ordernumber;
    '''
)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerNumber,customerName,contactLastName,...,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10425,2005-05-31,2005-06-07,None,In Process,None,119,119,La Rochelle Gifts,Labrune,...,None,44000,France,1370,118200.00,10100,S18_1749,30,136.00,3
1,10424,2005-05-31,2005-06-08,None,In Process,None,141,141,Euro+ Shopping Channel,Freyre,...,None,28034,Spain,1370,227600.00,10100,S18_1749,30,136.00,3
2,10423,2005-05-30,2005-06-05,None,In Process,None,314,314,Petit Auto,Dewey,...,None,B-1180,Belgium,1401,79900.00,10100,S18_1749,30,136.00,3
3,10422,2005-05-30,2005-06-11,None,In Process,None,157,157,Diecast Classics Inc.,Leong,...,PA,70267,USA,1216,100600.00,10100,S18_1749,30,136.00,3
4,10421,2005-05-29,2005-06-06,None,In Process,Custom shipping instructions were sent to ware...,124,124,Mini Gifts Distributors Ltd.,Nelson,...,CA,97562,USA,1165,210500.00,10100,S18_1749,30,136.00,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976691,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,None,141,141,Euro+ Shopping Channel,Freyre,...,None,28034,Spain,1370,227600.00,10425,S50_1392,18,94.92,2
976692,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,None,121,121,Baane Mini Imports,Bergulfsen,...,None,4110,Norway,1504,81700.00,10425,S50_1392,18,94.92,2
976693,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,None,181,181,Vitachrome Inc.,Frick,...,NY,10022,USA,1286,76400.00,10425,S50_1392,18,94.92,2
976694,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,128,"Blauer See Auto, Co.",Keitel,...,None,60528,Germany,1504,59700.00,10425,S50_1392,18,94.92,2


In [11]:
# bisa juga di sort based on requireddate u/ tau mana yg lbh urgent

query(
    '''
    SELECT c.customername, o.ordernumber, o.requireddate,od.productcode,od.quantityordered,priceeach,o.status,o.shippeddate
    FROM orders o
    JOIN customers c
	ON o.customernumber = c.customernumber
    JOIN orderdetails od
	ON o.ordernumber = o.ordernumber
    WHERE o.status != "shipped"
    ORDER BY o.requireddate
    '''
)

,customername,ordernumber,requireddate,productcode,quantityordered,priceeach,status,shippeddate
0,Scandinavian Gift Ideas,10167,2003-10-30,S18_4600,46,106.55,Cancelled,None
1,Mini Auto Werke,10164,2003-10-30,S18_4600,46,106.55,Resolved,2003-10-23
2,Scandinavian Gift Ideas,10167,2003-10-30,S18_4668,35,41.25,Cancelled,None
3,Mini Auto Werke,10164,2003-10-30,S18_4668,35,41.25,Resolved,2003-10-23
4,Scandinavian Gift Ideas,10167,2003-10-30,S32_1268,26,78.01,Cancelled,None
...,...,...,...,...,...,...,...,...
68903,Diecast Classics Inc.,10422,2005-06-11,S24_3191,42,69.34,In Process,None
68904,Diecast Classics Inc.,10422,2005-06-11,S24_3432,49,101.73,In Process,None
68905,Diecast Classics Inc.,10422,2005-06-11,S24_3969,20,39.80,In Process,None
68906,Diecast Classics Inc.,10422,2005-06-11,S18_2795,24,135.00,In Process,None


In [12]:
# u/ tau banyaknya produk dr setiap productline

query(
    '''
    SELECT productline, productcode, productname, productscale
    FROM products 
    ORDER BY productline, productcode;
    '''
)

,productline,productcode,productname,productscale
0,Classic Cars,S10_1949,1952 Alpine Renault 1300,1:10
1,Classic Cars,S10_4757,1972 Alfa Romeo GTA,1:10
2,Classic Cars,S10_4962,1962 LanciaA Delta 16V,1:10
3,Classic Cars,S12_1099,1968 Ford Mustang,1:12
4,Classic Cars,S12_1108,2001 Ferrari Enzo,1:12
...,...,...,...,...
105,Vintage Cars,S24_3816,1940 Ford Delivery Sedan,1:24
106,Vintage Cars,S24_3969,1936 Mercedes Benz 500k Roadster,1:24
107,Vintage Cars,S24_4258,1936 Chrysler Airflow,1:24
108,Vintage Cars,S32_4289,1928 Ford Phaeton Deluxe,1:32


In [13]:
# untuk tau hal yg harus di FU

query(
    '''
    SELECT C.customername, shippeddate, sum(quantityordered*priceeach) as TOTAL_YG_HRS_DIBAYAR
    FROM orders O
    JOIN orderdetails OD
	ON O.ordernumber = OD.ordernumber
    JOIN customers C
	ON o.customernumber = C.customernumber
    GROUP BY o.customernumber, shippeddate
    ORDER BY c.customername
;
    '''
)

,customername,shippeddate,TOTAL_YG_HRS_DIBAYAR
0,Alpha Cognac,2003-07-06,14232.70
1,Alpha Cognac,2003-11-10,33818.34
2,Alpha Cognac,2005-04-01,12432.32
3,Amica Models & Co.,2004-08-19,48298.99
4,Amica Models & Co.,2004-09-14,33924.24
...,...,...,...
318,"Volvo Model Replicas, Co",2004-11-07,16799.03
319,"Volvo Model Replicas, Co",2004-11-10,19206.68
320,"Volvo Model Replicas, Co",None,23014.17
321,West Coast Collectables Co.,2003-12-06,7678.25


In [14]:
# u/ tau banyaknya produk dr setiap productline

query(
    '''
    SELECT productline, productcode, productname, productscale
    FROM products 
    ORDER BY productline, productcode;
    '''
)

,productline,productcode,productname,productscale
0,Classic Cars,S10_1949,1952 Alpine Renault 1300,1:10
1,Classic Cars,S10_4757,1972 Alfa Romeo GTA,1:10
2,Classic Cars,S10_4962,1962 LanciaA Delta 16V,1:10
3,Classic Cars,S12_1099,1968 Ford Mustang,1:12
4,Classic Cars,S12_1108,2001 Ferrari Enzo,1:12
...,...,...,...,...
105,Vintage Cars,S24_3816,1940 Ford Delivery Sedan,1:24
106,Vintage Cars,S24_3969,1936 Mercedes Benz 500k Roadster,1:24
107,Vintage Cars,S24_4258,1936 Chrysler Airflow,1:24
108,Vintage Cars,S32_4289,1928 Ford Phaeton Deluxe,1:32


In [15]:
# u/ tau banyaknya produk dr setiap productline

query(
    '''
    SELECT productline, productcode, productname, productscale
    FROM products 
    ORDER BY productline, productcode;
    '''
)

,productline,productcode,productname,productscale
0,Classic Cars,S10_1949,1952 Alpine Renault 1300,1:10
1,Classic Cars,S10_4757,1972 Alfa Romeo GTA,1:10
2,Classic Cars,S10_4962,1962 LanciaA Delta 16V,1:10
3,Classic Cars,S12_1099,1968 Ford Mustang,1:12
4,Classic Cars,S12_1108,2001 Ferrari Enzo,1:12
...,...,...,...,...
105,Vintage Cars,S24_3816,1940 Ford Delivery Sedan,1:24
106,Vintage Cars,S24_3969,1936 Mercedes Benz 500k Roadster,1:24
107,Vintage Cars,S24_4258,1936 Chrysler Airflow,1:24
108,Vintage Cars,S32_4289,1928 Ford Phaeton Deluxe,1:32


In [17]:


-- untuk tau hal yg harus di FU
select distinct comments from orders;
select orders.comments, customers.customerName, customers.creditLimit  
from orders, customers
where comments like '%limit exceed%' and customers.customerNumber = orders.customerNumber;

-- BARU u/ tau yg barus dibayar
SELECT C.customername, shippeddate, sum(quantityordered*priceeach) as TOTAL_YG_HRS_DIBAYAR
FROM orders O
JOIN orderdetails OD
	ON O.ordernumber = OD.ordernumber
JOIN customers C
	ON o.customernumber = C.customernumber
GROUP BY o.customernumber, shippeddate
order by c.customername
;

-- BARU untuk tahu udh lebih dr credit limit atau blm
SELECT 
	P.customernumber, 
	C.customername, 
    O.shippeddate,
    sum(quantityordered*priceeach) as TOTAL_YG_HRS_DIBAYAR, 
    (SELECT sum(amount) from payments GROUP BY paymentdate ) AS YG_UDH_DIBAYAR,
    (sum(quantityordered*priceeach)-sum(P.amount)) AS BALANCE_CREDIT, 
    C.creditlimit 
FROM PAYMENTS P
JOIN customers C 
	ON C.customernumber = P.customernumber
JOIN orders O
 	ON O.customernumber = P.customernumber
JOIN orderdetails OD
	ON O.ordernumber = OD.ordernumber
GROUP BY P.customernumber
;

-- list karyawan di tiap kantor
select offices.city ,employees.firstName, employees.lastName, employees.jobTitle
from employees, offices
where offices.officeCode = employees.officeCode;

-- nampilin produk terlaris
select orderdetails.productcode, products.productname, orderdetails.quantityOrdered 
from orderdetails,products 
where orderdetails.productcode = products.productcode 
group by productCode 
order by quantityordered desc;

-- nampilin jumlah qty order
select products.productline, sum(orderdetails.quantityOrdered) as total
from orderdetails,products 
where orderdetails.productcode = products.productcode 
group by productline
order by total desc;


-- total pendapatan
select paymentDate, sum(amount) from payments group by paymentdate order by paymentdate;

-- stock plg banyak & min
select productname, quantityinstock from products order by quantityinstock desc limit 10;
select productname, quantityinstock from products order by quantityinstock limit 10;

-- product vendor / supplier 
select distinct productvendor from products;
select count(distinct productvendor) from products;

-- u/ tau customer yg udh pernah belanja
SELECT
	customers.contactFirstName,
    customers.contactLastName, 
    orders.orderdate,
    orders.orderNumber
FROM
	customers,orders
WHERE customers.customernumber = orders.customernumber
ORDER BY customers.contactFirstName;

-- u/ tau supplier apa bikin apa aja
select productvendor, productname, productline from products order by productvendor;

-- untuk tau margin profit terbesar
select products.productCode, products.productName, products.buyprice, orderdetails.priceeach, orderdetails.priceeach - products.buyprice as MarginProfit
from orderdetails, products
where products.productCode = orderdetails.productCode
-- group by productcode
order by productCode;

select * from orderdetails order by productCode;
select count(distinct productLine) from products;
select distinct productLine from products;
select productName, productLine, productVendor from products where productLine = 'Classic Cars' limit 10;
select * from products where productLine = 'Planes';
select count(productName) from products where productLine = 'Planes';
select * from productlines where productLIne = 'Planes';
select * from orderdetails where productCode = (select productCode from productlines where productLIne = 'Planes');
-- untuk tahu orderan pesawat ada apa aja
select sum(quantityOrdered) from orderdetails where productCode = (select productCode from productlines where productLIne = 'Planes');
-- qty order pesawat
select ordernumber, productCode, quantityOrdered,priceEach, quantityOrdered*priceEach as Total from orderdetails where productCode = (select productCode from productlines where productLine = 'Planes');
-- nampilin total dr qty * harga
select customerNumber from orders where orderNumber = 10100;
-- hasilnya 363
select * from customers where customerNumber = 363;
-- nampilin detail customer 363
select * from orders where customerNumber = 363;
-- nampilin pesanan si customer 363 apa aja ; 10100, 10192, 10322
select ordernumber, productCode, quantityOrdered,priceEach, quantityOrdered*priceEach as Total from orderdetails where orderNumber = 10100 or orderNumber = 10192 or orderNumber = 10322;

select count(distinct productLine) from products;
select distinct productLine from products;
select productName, productLine, productVendor from products where productLine = 'Classic Cars' limit 10;
select * from payments;


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 22)